In [14]:
import numpy as np
import pandas as pd
from numpy import array
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [15]:
population_df = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/AnalyseProject/world_population.csv', index_col='Country Code')
meta_df = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/AnalyseProject/metadata.csv', index_col='Country Code')

In [16]:
population_df.head()

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Country Code,,,,,,,,,,,,,,,,,,,,,
ABW,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,57715.0,58055.0,58386.0,58726.0,...,101353.0,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822.0,105264.0
AFG,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,9938414.0,10152331.0,10372630.0,10604346.0,10854428.0,...,27294031.0,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032.0,35530081.0
AGO,5643182.0,5753024.0,5866061.0,5980417.0,6093321.0,6203299.0,6309770.0,6414995.0,6523791.0,6642632.0,...,21759420.0,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0,28813463.0,29784193.0
ALB,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,1914573.0,1965598.0,2022272.0,2081695.0,...,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0
AND,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,19647.0,20758.0,21890.0,23058.0,...,83861.0,84462.0,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0,77281.0,76965.0


In [17]:
meta_df.head()

,Region,Income Group,Special Notes
Country Code,,,
ABW,Latin America & Caribbean,High income,Mining is included in agriculture\r\r\r\nElect...
AFG,South Asia,Low income,Fiscal year end: March 20; reporting period fo...
AGO,Sub-Saharan Africa,Lower middle income,NaN
ALB,Europe & Central Asia,Upper middle income,NaN
AND,Europe & Central Asia,High income,WB-3 code changed from ADO to AND to align wit...


In [29]:
print(meta_df.columns)

Index(['Region', 'Income Group', 'Special Notes'], dtype='object')


In [30]:
print(population_df.columns)


Index(['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017'],
      dtype='object')


In [18]:
filtered_df = meta_df[meta_df['Income Group'] == 'Low income']
filtered_df.head()

,Region,Income Group,Special Notes
Country Code,,,
AFG,South Asia,Low income,Fiscal year end: March 20; reporting period fo...
BDI,Sub-Saharan Africa,Low income,NaN
BEN,Sub-Saharan Africa,Low income,NaN
BFA,Sub-Saharan Africa,Low income,NaN
CAF,Sub-Saharan Africa,Low income,The source of national accounts has changed to...


In [19]:
print(filtered_df.columns)

Index(['Region', 'Income Group', 'Special Notes'], dtype='object')


In [34]:
indexes = filtered_df.index.tolist()
print(indexes)

['AFG', 'BDI', 'BEN', 'BFA', 'CAF', 'COD', 'COM', 'ERI', 'ETH', 'GIN', 'GMB', 'GNB', 'HTI', 'LBR', 'MDG', 'MLI', 'MOZ', 'MWI', 'NER', 'NPL', 'PRK', 'RWA', 'SEN', 'SLE', 'SOM', 'SSD', 'SYR', 'TCD', 'TGO', 'TJK', 'TZA', 'UGA', 'YEM', 'ZWE']


In [32]:
# Extract population data for the specified country
country_data = filtered_df.loc[filtered_df.index == 'AFG']
print(country_data)

                  Region Income Group  \
Country Code                            
AFG           South Asia   Low income   

                                                  Special Notes  
Country Code                                                     
AFG           Fiscal year end: March 20; reporting period fo...  


In [37]:
# Extracting data for specific indices along with all years
data_from_other_df = population_df.loc[indexes]
print(data_from_other_df)


                    1960        1961        1962        1963        1964  \
Country Code                                                               
AFG            8996351.0   9166764.0   9345868.0   9533954.0   9731361.0   
BDI            2786106.0   2839666.0   2893669.0   2949926.0   3010859.0   
BEN            2431622.0   2465867.0   2502896.0   2542859.0   2585965.0   
BFA            4829288.0   4894580.0   4960326.0   5027821.0   5098890.0   
CAF            1503508.0   1529227.0   1556661.0   1585763.0   1616516.0   
COD           15248251.0  15637733.0  16041263.0  16461930.0  16903923.0   
COM             191121.0    194139.0    197198.0    200372.0    203753.0   
ERI            1397491.0   1432640.0   1469645.0   1508273.0   1548187.0   
ETH           22151278.0  22671190.0  23221389.0  23798429.0  24397024.0   
GIN            3577409.0   3633652.0   3690664.0   3749505.0   3811659.0   
GMB             367928.0    376737.0    383523.0    389072.0    394553.0   
GNB         

In [39]:
# Assuming 'other_df' is the DataFrame with years as columns and population as rows
# 'indexes' contains the indices obtained from 'filtered_df'

# Extracting data for specific indices along with all years as an array without indices
data_array = population_df.loc[indexes].values.T
print(data_array)

[[ 8996351.  2786106.  2431622. ...  6788214.  5172135.  3747369.]
 [ 9166764.  2839666.  2465867. ...  7006633.  5260501.  3870756.]
 [ 9345868.  2893669.  2502896. ...  7240174.  5351799.  3999419.]
 ...
 [33736494. 10199270. 10575952. ... 40144870. 26916207. 15777451.]
 [34656032. 10524117. 10872298. ... 41487965. 27584213. 16150362.]
 [35530081. 10864245. 11175692. ... 42862958. 28250420. 16529904.]]


In [41]:
import numpy as np

# Assuming 'other_df' is the DataFrame with years as columns and population as rows
# 'indexes' contains the indices obtained from 'filtered_df'

# Extracting data for specific indices along with all years as an array without indices
data_array = population_df.loc[indexes].values

# Reshape the array to have two columns (year, population) and transpose it
reshaped_array = np.column_stack((population_df.columns, data_array.T))

# Checking the shape
print(reshaped_array)  


[['1960' 8996351.0 2786106.0 ... 6788214.0 5172135.0 3747369.0]
 ['1961' 9166764.0 2839666.0 ... 7006633.0 5260501.0 3870756.0]
 ['1962' 9345868.0 2893669.0 ... 7240174.0 5351799.0 3999419.0]
 ...
 ['2015' 33736494.0 10199270.0 ... 40144870.0 26916207.0 15777451.0]
 ['2016' 34656032.0 10524117.0 ... 41487965.0 27584213.0 16150362.0]
 ['2017' 35530081.0 10864245.0 ... 42862958.0 28250420.0 16529904.0]]


In [ ]:
### START FUNCTION
def get_total_pop_by_income(income_group_name='Low income'):
    
    return

### END FUNCTION